In [1]:
import numpy as np
import cv2

In [1]:
def remap(img, x, y):
    x, y = np.float32(x), np.float32(y)
    out = cv2.remap(img, x, y, cv2.INTER_LINEAR)  # INTER_LANCZOS4 INTER_CUBIC INTER_LINEAR
    return out

def cdi2fdi(u_cdi, v_cdi, delta=1):
    assert u_cdi.shape == v_cdi.shape
    # shape[0]:height, shape[1]:width
    y, x = np.meshgrid(np.arange(u_cdi.shape[0]), np.arange(u_cdi.shape[1]),indexing='ij')
    u_c, v_c = u_cdi/delta, v_cdi/delta
    
    # init fdi field for later process
    u_f = remap(u_c, x+0.5*u_c, y+0.5*v_c)
    v_f = remap(v_c, x+0.5*u_c, y+0.5*v_c)
    
    # solve 'Vcdi[x + 1/2Vfdi(x)] = Vfdi(x)' via iteration
    for _ in range(5):
        u_f = remap(u_c, x+0.5*u_f, y+0.5*v_f)
        v_f = remap(v_c, x+0.5*u_f, y+0.5*v_f)
    u_f, v_f = delta*u_f, delta*v_f
    return u_f, v_f